In [4]:
names = ['thorax', 'neck', 'a_L1', 'a_L2', 'a_R1', 'a_R2', 'leg_f_L1', 'leg_f_L2', 'leg_f_R1', 'leg_f_R2', 'leg_m_L1', 'leg_m_L2', 'leg_m_R1', 'leg_m_R2', 'a_R0', 'a_L0', 'leg_f_R0', 'leg_f_L0', 'leg_m_R0', 'leg_m_L0', 'eye_L', 'eye_R', 'm_L0', 'm_R0', 'm_L1', 'm_R1']

In [1]:
from src.collision import CollisionDetector
# t = CollisionDetector(obj_folder="./example_obj_folder/", skeleton_json_path="", pose_csv="ant_3d.csv")
t = CollisionDetector(obj_folder="./example_obj_folder/seed/", skeleton_json_path="", pose_csv="1616_session28_ant_3d_WITHFRAMES.csv")


In [2]:
from src.visual_widget import Viewer
V = Viewer(t.animal, t.obj, 1000)

In [5]:
s=V.scene_widget.scene.geometry